In [6]:
import json

def analyze_build_results(filename):
    """
    Reads a JSON file and provides a complete breakdown of build and test outcomes.

    Returns:
        tuple: (summary_dict, individual_counts_dict)
    """
    successful_builds = 0
    failed_builds = 0
    builds_with_failing_tests = 0
    builds_with_passing_tests = 0
    
    # This dictionary will store counts for all combinations of build_ok and return_code.
    individual_counts = {}

    try:
        with open(filename, 'r') as file:
            data = json.load(file)

        if 'results' in data and isinstance(data['results'], list):
            for result in data['results']:
                build_ok = result.get('build_ok')
                return_code = result.get('return_code')
                
                # Use a tuple for the key to get individual outcome counts
                key = (build_ok, return_code)
                individual_counts[key] = individual_counts.get(key, 0) + 1

                # Tally builds based on their outcome
                if build_ok is True:
                    if return_code == 0:
                        builds_with_passing_tests += 1
                        successful_builds += 1
                    else:
                        builds_with_failing_tests += 1
                        # Note: builds with failing tests are still "successful" in terms of build_ok=true
                        # but are not completely successful. This logic separates them.
                else:
                    failed_builds += 1
        else:
            print("Error: JSON file does not contain a 'results' list.")

    except FileNotFoundError:
        print(f"Error: The file '{filename}' was not found.")
        return {}, {}
    except json.JSONDecodeError:
        print(f"Error: The file '{filename}' contains invalid JSON.")
        return {}, {}
    
    total_builds = failed_builds + builds_with_failing_tests + builds_with_passing_tests
    
    summary = {
        'total_builds': total_builds,
        'successful_builds': successful_builds,
        'failed_builds': failed_builds,
        'builds_with_passing_tests': builds_with_passing_tests,
        'builds_with_failing_tests': builds_with_failing_tests
    }
    
    return summary, individual_counts



In [7]:

file_name = 'docker_test_summary.json'
successful_count, individual_counts = analyze_build_results(file_name)

# Print the results
print(f"Total successful builds (build_ok=true and return_code=0): {successful_count}")
print("\nIndividual counts for each outcome:")
for (build_ok, return_code), count in individual_counts.items():
    print(f"  build_ok={build_ok}, return_code={return_code}: {count}")



Total successful builds (build_ok=true and return_code=0): {'total_builds': 211, 'successful_builds': 38, 'failed_builds': 102, 'builds_with_passing_tests': 38, 'builds_with_failing_tests': 71}

Individual counts for each outcome:
  build_ok=False, return_code=0: 102
  build_ok=True, return_code=0: 38
  build_ok=True, return_code=1: 13
  build_ok=True, return_code=4: 58
